https://towardsdatascience.com/simple-ocr-with-tesseract-a4341e4564b6

In [1]:
import os
import shutil
from PIL import Image
from matplotlib import pyplot as plt
import pytesseract
import numpy as np
import cv2

In [2]:
shutil.rmtree('train')

In [3]:
font = list(range(0x7f))
for a in range(0x20, 0x7f):
    font[a] = Image.open("font/" + hex(a)[2:].upper() + ".png")

In [4]:
def gen_image(text):
    pad = 4
    width = 0
    for a in text:
        width += font[ord(a)].size[0] + 1
    width += pad*2
    height = font[ord('A')].size[1] + pad*2
    img = Image.new(mode = "RGB", size = (width, height), color = (255, 255, 255))
    
    boxes = []
    left = pad
    for a in text:
        letter = font[ord(a)]
        top = height - letter.height - pad
        img.paste(letter, (left, pad))
        boxes.append((a, left, pad - 2, left + letter.width - 1, height - pad + 2))
        left += letter.size[0] + 1
    return (img, boxes)

In [5]:
text = ''
for a in range(0x21, 0x7f):
    text = text + chr(a)
text

'!"#$%&\'()*+,-./0123456789:;<=>?@ABCDEFGHIJKLMNOPQRSTUVWXYZ[\\]^_`abcdefghijklmnopqrstuvwxyz{|}~'

In [6]:
gen, boxes = gen_image(text)
gen

In [7]:
%time pytesseract.image_to_string(gen)

CPU times: user 5.45 ms, sys: 17.7 ms, total: 23.2 ms
Wall time: 487 ms


"WH$BS'Cy*+,-. 0123456789:\n\n  \n\n-? @ABCDEFGHIJIKLMNOPORSTUYWXYZDN1*_‘abedef ghijkimnopgrstuvwxyz{]}”\n"

In [8]:
boxes_lines = []
#for params in boxes:
#    boxes_lines.append('%s %d %d %d %d %d' % (*params, 0))

Generate other size images

In [9]:
os.makedirs("train", exist_ok=True)

In [10]:
pages = []
page_ind = -1

def add_page(sf, mode):
    global page_ind
    page_ind += 1
    im = gen.resize((gen.width*sf, gen.height*sf), mode)
    pages.append(im)
    for params in boxes:
        (c,l,t,r,b) = params
        boxes_lines.append('%s %d %d %d %d %d' % (c, l*sf, t*sf, r*sf, b*sf, page_ind))

In [11]:
add_page(2, Image.ANTIALIAS)
add_page(2, Image.NEAREST)
add_page(3, Image.ANTIALIAS)
add_page(3, Image.NEAREST)
add_page(4, Image.ANTIALIAS)
add_page(4, Image.NEAREST)

In [12]:
with open('train/lb.font.exp0.box', 'w') as f:
    for line in boxes_lines:
        f.write(line)
        f.write('\n')

In [13]:
# Merge tif
#gen.save("train/lb.font.exp0.tif", format="tiff", append_images=pages, save_all=True)
pages[0].save("train/lb.font.exp0.tif", format="tiff", append_images=pages[1:], save_all=True)

In [14]:
with open('train/font_properties', 'w') as f:
    f.write("font 0 0 0 0 0")

In [15]:
%cd train

/home/jovyan/work/train


In [16]:
# Create a .tr file (training file)
!tesseract lb.font.exp0.tif lb.font.exp0 nobatch box.train

Page 1
Estimating resolution as 210
row xheight=3, but median xheight = 4
row xheight=10.9086, but median xheight = 4
FAIL!
APPLY_BOXES: boxfile line 29/= ((370,4),(384,28)): FAILURE! Couldn't find a matching blob
APPLY_BOXES:
   Boxes read from boxfile:      94
   Boxes failed resegmentation:       1
APPLY_BOXES: Unlabelled word at :Bounding box=(352,12)->(356,18)
   Found 93 good blobs.
   1 remaining unlabelled words deleted.
Generated training data for 3 words
Page 2
Estimating resolution as 210
row xheight=3, but median xheight = 4
row xheight=10.9086, but median xheight = 4
FAIL!
APPLY_BOXES: boxfile line 29/= ((370,4),(384,28)): FAILURE! Couldn't find a matching blob
APPLY_BOXES:
   Boxes read from boxfile:      94
   Boxes failed resegmentation:       1
APPLY_BOXES: Unlabelled word at :Bounding box=(352,12)->(356,18)
   Found 93 good blobs.
   1 remaining unlabelled words deleted.
Generated training data for 3 words
Page 3
Estimating resolution as 331
APPLY_BOXES:
   Boxes read

In [17]:
# Create a unicharset file
!unicharset_extractor lb.font.exp0.box

Extracting unicharset from box file lb.font.exp0.box
Wrote unicharset file unicharset


In [23]:
!combine_lang_model

USAGE: combine_lang_model
  --lang_is_rtl  True if lang being processed is written right-to-left  (type:bool default:false)
  --pass_through_recoder  If true, the recoder is a simple pass-through of the unicharset. Otherwise, potentially a compression of it  (type:bool default:false)
  --input_unicharset  Filename with unicharset to complete and use in encoding  (type:string default:)
  --script_dir  Directory name for input script unicharsets  (type:string default:)
  --words  File listing words to use for the system dictionary  (type:string default:)
  --puncs  File listing punctuation patterns  (type:string default:)
  --numbers  File listing number patterns  (type:string default:)
  --output_dir  Root directory for output files  (type:string default:)
  --version_str  Version string to add to traineddata file  (type:string default:)
  --lang  Name of language being processed  (type:string default:)


In [21]:
!lstmtraining -F font_properties -U unicharset -O lb.unicharset lb.font.exp0.tr

Must provide a --traineddata see training documentation
